# Data Cleaning

## Data Types

 - Numeric
 - Categorical
 - Binary
 - Ordinal
 - Dates

https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes

In [3]:
import pandas as pd
import numpy as np


In [5]:
data = '.\\data\\dirty_cafe_sales.csv'
df = pd.read_csv(data)

In [13]:
df.tail(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
9990,TXN_1538510,Coffee,5,2.0,10.0,Digital Wallet,NaN,2023-05-22
9991,TXN_3897619,Sandwich,3,4.0,12.0,Cash,Takeaway,2023-02-24
9992,TXN_2739140,Smoothie,4,4.0,16.0,UNKNOWN,In-store,2023-07-05
9993,TXN_4766549,Smoothie,2,4.0,NaN,Cash,NaN,2023-10-20
9994,TXN_7851634,UNKNOWN,4,4.0,16.0,NaN,NaN,2023-01-08
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02
9999,TXN_6170729,Sandwich,3,4.0,12.0,Cash,In-store,2023-11-07


## Duplicates

Helpful methods: <br>
- unique()
- nunique()
- is_duplicated()
- drop_duplicates()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [21]:
df['Item'].unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'UNKNOWN',
       'Sandwich', nan, 'ERROR', 'Juice', 'Tea'], dtype=object)

In [23]:
df['Item'].nunique()

10

In [25]:
df['Payment Method'].nunique()

5

In [27]:
df['Location'].nunique()

4

In [31]:
df['Transaction ID'].nunique(), df.shape[0]

(10000, 10000)

## Working with NULL values

In [80]:
df.isna().sum().sort_values(ascending=False)#.reset_index()

Location            3265
Payment Method      2579
Item                 333
Price Per Unit       179
Total Spent          173
Transaction Date     159
Quantity             138
Transaction ID         0
dtype: int64

Reasons for NULL values:
<br>
- Not Exist
- Not Recorded

In [35]:
df.isna().sum().sum() / df.shape[0] * df.shape[1]

5.4608

### Solutions

If missing values are > 75% then drop, else fill them in.

#### Dropping

In [39]:
df.dropna(subset=['Item', 'Location'])

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9986,TXN_2858441,Sandwich,2,4.0,8.0,Credit Card,In-store,2023-12-14
9991,TXN_3897619,Sandwich,3,4.0,12.0,Cash,Takeaway,2023-02-24
9992,TXN_2739140,Smoothie,4,4.0,16.0,UNKNOWN,In-store,2023-07-05
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30


https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html

#### Automatic Filling

In [ ]:
df.fillna(0)

`pandas.DataFrame.fillna()` has one important argument if value is not given: **method**<br>
- ffill: propagate last valid observation forward to next valid.
- bfill: use next valid observation to fill gap.

#### Better Filling

Filling with statistics. For examples, `mean`, `mode` or `median`.<br>
 - sklearn's `SimpleImputer` method.

In [45]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit([[1, 2], [np.nan, 3], [7, 6]])
X = [[np.nan, 2], [6, np.nan], [7, 6]]
print(imp.transform(X))

[[4.         2.        ]
 [6.         3.66666667]
 [7.         6.        ]]


In [67]:
df['Total Spent'].value_counts()

Total Spent
6.0     979
12.0    939
3.0     930
4.0     923
20.0    746
15.0    734
8.0     677
10.0    524
2.0     497
9.0     479
5.0     468
16.0    444
25.0    259
7.5     237
1.0     232
4.5     225
1.5     205
Name: count, dtype: int64

In [65]:
df.loc[df['Total Spent'].isin(['UNKNOWN', 'ERROR']), 'Total Spent'] = np.nan

In [73]:
df['Total Spent'] = df['Total Spent'].astype('float')
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(pd.DataFrame(df['Total Spent']))

print(imp.transform(pd.DataFrame(df['Total Spent'])))

[[ 4.       ]
 [12.       ]
 [ 8.9243525]
 ...
 [ 8.       ]
 [ 3.       ]
 [12.       ]]


#### Can we do even better?

## Outlier Detection

#### z-score

 This approach measures how many standard deviations a data point is from the mean. Data points beyond a certain threshold (e.g., ±3 standard deviations) can be considered outliers.

In [ ]:

from scipy import stats 
z_scores = stats.zscore(data['column']) outliers = abs(z_scores) > 3 # Identifying outliers

#### IQR (Interquartile) range

IQR is the range between the first quartile (Q1) and the third quartile (Q3). Values beyond 1.5 times the IQR above Q3 or below Q1 are considered outliers.

In [ ]:
Q1 = data['column'].quantile(0.25)
Q3 = data['column'].quantile(0.75)
IQR = Q3 - Q1
outliers = (data['column'] < (Q1 - 1.5 * IQR)) | (data['column'] > (Q3 + 1.5 * IQR))

# BONUS

In [ ]:
def reduce_mem_usage(df, cat_cols=[]):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    # print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    for cat in cat_cols:
        df[cat] = df[cat].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    # print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    # print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df